In [4]:
import numpy as np
import h5py

In [6]:
import random
from tqdm import tqdm

def generate_packed_spheres(num_spheres, radius, box_size, max_attempts_per_sphere=1000):
    """
    Generates a packing of non-overlapping spheres within a rectangular box
    with integer sphere positions, showing a progress bar.

    Args:
        num_spheres (int): The desired number of spheres.
        radius (float): The radius of each sphere.
        box_size (tuple): A tuple (Nx, Ny, Nz) representing the integer
                          dimensions of the bounding box.
        max_attempts_per_sphere (int): Maximum attempts to place each sphere.

    Returns:
        list: A list of sphere positions (center coordinates as integer NumPy arrays)
              if successful, otherwise an empty list.
    """
    Nx, Ny, Nz = box_size  # Unpack the box dimensions
    sphere_positions = []

    # Ensure the radius is compatible with integer positions and a non-empty box
    if not isinstance(radius, int):
        print("Warning: Radius should ideally be an integer when aiming for integer sphere positions to avoid unexpected behavior.")

    # Calculate the valid integer range for sphere centers
    min_coord = int(radius)
    max_x = int(Nx - radius)
    max_y = int(Ny - radius)
    max_z = int(Nz - radius)

    if min_coord >= max_x or min_coord >= max_y or min_coord >= max_z:
        print("Error: Radius is too large for the specified box size, or box size is too small.")
        return []

    # Wrap the main loop with tqdm to display a progress bar
    for _ in tqdm(range(num_spheres), desc="Packing spheres"):
        placed = False
        attempts = 0
        while not placed and attempts < max_attempts_per_sphere:
            # Generate random integer positions within the valid range
            new_x = random.randint(min_coord, max_x)
            new_y = random.randint(min_coord, max_y)
            new_z = random.randint(min_coord, max_z)
            new_pos = np.array([new_x, new_y, new_z], dtype=int)  # Ensure integer type

            # Check for overlaps with existing spheres
            overlap = False
            for existing_pos in sphere_positions:
                distance = np.linalg.norm(new_pos - existing_pos)
                if distance < 2 * radius:  # Overlap if distance is less than sum of radii
                    overlap = True
                    break

            if not overlap:
                sphere_positions.append(new_pos)
                placed = True
            else:
                attempts += 1

        if not placed:
            print(f"Could not place all spheres. Placed {len(sphere_positions)} out of {num_spheres}.")
            break

    return sphere_positions


def generate_sphere_array(radius, sphere_positions, box_size):
    """
    Generates a 3D NumPy array representing the packed spheres.

    The array is initially filled with ones (representing empty space),
    and then voxels within each sphere are set to zero.

    Args:
        radius (float): The radius of the spheres.
        sphere_positions (list): A list of sphere positions (center coordinates)
                                 as NumPy arrays (e.g., generated by generate_packed_spheres).
        box_size (tuple): A tuple (Nx, Ny, Nz) representing the dimensions of the box.

    Returns:
        numpy.ndarray: A 3D NumPy array with spheres marked as zeros.
    """
    Nx, Ny, Nz = box_size

    # Initialize the 3D array with ones
    sphere_array = np.ones((Nx, Ny, Nz), dtype=int)

    # Pre-calculate squared radius for faster distance checks
    radius_squared = radius**2

    # Iterate through each sphere and mark the voxels within it
    for center_x, center_y, center_z in sphere_positions:
        # Determine the bounding box for the sphere's influence to optimize iteration
        min_x = max(0, int(center_x - radius))
        max_x = min(Nx, int(center_x + radius) + 1)
        min_y = max(0, int(center_y - radius))
        max_y = min(Ny, int(center_y + radius) + 1)
        min_z = max(0, int(center_z - radius))
        max_z = min(Nz, int(center_z + radius) + 1)

        # Loop through the relevant voxels
        for i in range(min_x, max_x):
            for j in range(min_y, max_y):
                for k in range(min_z, max_z):
                    # Calculate the distance from the voxel's center to the sphere's center
                    # We assume voxel (i,j,k) corresponds to coordinates (i+0.5, j+0.5, k+0.5)
                    # For integer sphere centers and integer radius, this is slightly simplified
                    # as (i-center_x)^2 + (j-center_y)^2 + (k-center_z)^2
                    distance_squared = (i - center_x)**2 + \
                                       (j - center_y)**2 + \
                                       (k - center_z)**2

                    # If the voxel is inside or on the boundary of the sphere, mark it as 0
                    if distance_squared <= radius_squared:
                        sphere_array[i, j, k] = 0

    return sphere_array


In [ ]:
Nx = 50; Ny = 50; Nz = 200
box_dimensions = (Nx, Ny, Nz)
sphere_radius = 10
num_spheres_to_place = 10000
add_padding = True

packed_spheres = generate_packed_spheres(num_spheres_to_place, sphere_radius, box_dimensions, max_attempts_per_sphere = 100000)

if packed_spheres:
    voxels = generate_sphere_array(sphere_radius, packed_spheres, box_dimensions)

    print(f"Successfully placed {len(packed_spheres)} spheres:")
else:
    print("Failed to place spheres.")

num_zeros = len(np.where(voxels == 0)[0])
packing_efficiency = num_zeros / (Nx * Ny * Nz)
print(f"Packing efficiency: {packing_efficiency * 100}")

voxels_padded = voxels
if add_padding:
  padding = ((0, 0), (0, 0), (20, 20))
  voxels_padded = np.pad(voxels, pad_width=padding, mode='constant', constant_values=1)

print(voxels_padded.shape)

# voxels_1D = np.reshape(voxels_padded, [voxels_padded.shape[0] * voxels_padded.shape[1] * voxels_padded.shape[2], 1], order='C')
# np.savetxt('pebbles.dat', voxels_1D, fmt='%i')

with h5py.File("binary_media.h5", "w") as hdf5_file:
    hdf5_file.create_dataset("binary_media", data = voxels_padded.transpose())


Packing spheres:   0%|          | 35/10000 [00:03<15:47, 10.52it/s] 

Could not place all spheres. Placed 35 out of 10000.
Successfully placed 35 spheres:
Packing efficiency: 29.181400000000004
(50, 50, 240)
